## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

D:\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3,3), 
                                 activation='relu',
                                 input_shape=(32,32,3)))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

# 池化
classifier.add(MaxPooling2D(pool_size=(2, 2)))

#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3,3)))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=10)

W1212 23:55:46.302493 25052 deprecation_wrapper.py:119] From D:\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1212 23:55:46.321483 25052 deprecation_wrapper.py:119] From D:\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1212 23:55:46.324480 25052 deprecation_wrapper.py:119] From D:\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1212 23:55:46.368463 25052 deprecation_wrapper.py:119] From D:\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W1212 23:55:46.370457 25052 deprecation_wrapper.py:119] From D:\Anaconda3\lib\site-packages\keras\backen

Epoch 1/10
50000/50000 [==============================] - 112s 2ms/step - loss: 1.3932 - acc: 0.5371
Epoch 2/10
50000/50000 [==============================] - 101s 2ms/step - loss: 0.8859 - acc: 0.6932
Epoch 3/10
50000/50000 [==============================] - 101s 2ms/step - loss: 0.7024 - acc: 0.7553
Epoch 4/10
50000/50000 [==============================] - 101s 2ms/step - loss: 0.5622 - acc: 0.8044
Epoch 5/10
50000/50000 [==============================] - 101s 2ms/step - loss: 0.4456 - acc: 0.8443
Epoch 6/10
50000/50000 [==============================] - 102s 2ms/step - loss: 0.3481 - acc: 0.8788
Epoch 7/10
50000/50000 [==============================] - 102s 2ms/step - loss: 0.2765 - acc: 0.9040
Epoch 8/10
50000/50000 [==============================] - 103s 2ms/step - loss: 0.2135 - acc: 0.9244
Epoch 9/10
50000/50000 [==============================] - 101s 2ms/step - loss: 0.1636 - acc: 0.9436
Epoch 10/10
50000/50000 [==============================] - 101s 2ms/step - loss: 0.1532 - a

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[5.1178671e-02, 2.7247750e-06, 6.4809355e-03, 9.0264356e-01,
        9.1183539e-03, 1.5255988e-03, 8.0356537e-04, 3.5291305e-04,
        2.7888749e-02, 4.8161301e-06]], dtype=float32)